# Track 2.2 Outline a potential growth pipeline
- Build an analytics pipeline that shows us what job openings are related to machine learning or AI so how we could cluster job openings related to that kind of domain.
- Logic Guide:
    - We pick out the columns that most likely describe whether the job is AI/ML related;
    - Then we do a quick and raw filtering by checking if the job description mentioned AI/ML related phrase at all, this bring us down to only 3554 rows;
    - For each column, if it has AI/ML relevant information, we give it some score. This puts more weights on the rows that are likely to be AI/ML related jobs;
    - We add up the scores for each column and get a final score for the row. Then we sort it by descending order;
    - We need to observe and choose a cutting line to eliminate the least relevant rows.

In [1]:
# Import libraries
import pandas as pd
import numpy as np

### Step 1 : We select all the datasets that could be related to revealing more information about the job
- Selecting the most relevant columns;
- Lower-casing all the strings so that we don't double-cout or missing any information;
- joining the tables.

In [2]:
# Read the postings data
postings_df = pd.read_csv("data/postings.csv")
company_industries_df = pd.read_csv("data/companies/company_industries.csv")
company_specialities_df = pd.read_csv("data/companies/company_specialities.csv")
job_industries_df = pd.read_csv("data/jobs/job_industries.csv")
job_skills_df = pd.read_csv("data/jobs/job_skills.csv")
industries_df = pd.read_csv("data/mappings/industries.csv")
skills_df = pd.read_csv("data/mappings/skills.csv")

In [3]:
# Checking all columns that would be related to giving information about AI/ML related jobs
postings_df.columns

Index(['job_id', 'company_name', 'title', 'description', 'max_salary',
       'pay_period', 'location', 'company_id', 'views', 'med_salary',
       'min_salary', 'formatted_work_type', 'applies', 'original_listed_time',
       'remote_allowed', 'job_posting_url', 'application_url',
       'application_type', 'expiry', 'closed_time',
       'formatted_experience_level', 'skills_desc', 'listed_time',
       'posting_domain', 'sponsored', 'work_type', 'currency',
       'compensation_type'],
      dtype='object')

In [4]:
# Keep the wanted columns
filtered_posting_df = postings_df[['job_id','company_id','description','title','skills_desc']]
filtered_posting_df.head(20)

,job_id,company_id,description,title,skills_desc
0,921716,2774458.0,Job descriptionA leading real estate firm in N...,Marketing Coordinator,Requirements: \n\nWe are seeking a College or ...
1,1829192,NaN,"At Aspen Therapy and Wellness , we are committ...",Mental Health Therapist/Counselor,NaN
2,10998357,64896719.0,The National Exemplar is accepting application...,Assitant Restaurant Manager,We are currently accepting resumes for FOH - A...
3,23221523,766262.0,Senior Associate Attorney - Elder Law / Trusts...,Senior Elder Law / Trusts and Estates Associat...,This position requires a baseline understandin...
4,35982263,NaN,Looking for HVAC service tech with experience ...,Service Technician,NaN
5,91700727,1481176.0,Job summary:The Economic Development & Plannin...,Economic Development and Planning Intern,NaN
6,103254301,81942316.0,Company DescriptionRaw Cereal is a creative de...,Producer,NaN
7,112576855,NaN,Summary: Due to the pending retirement of our ...,Building Engineer,NaN
8,1218575,721189.0,"At Children’s, the region’s only full-service ...",Respiratory Therapist,• Requires the ability to communicate effectiv...
9,2264355,28631247.0,It is an exciting time to be a part of our chu...,Worship Leader,"Knowledge, Skills and Abilities: 1. Proficient..."


In [5]:
# Make all the strings in columns lowercase so that we are not missing/double-counting any information
job_skills_df['skill_abr'] = job_skills_df['skill_abr'].str.lower()
skills_df['skill_abr'] = skills_df['skill_abr'].str.lower()
skills_df['skill_name'] = skills_df['skill_name'].str.lower()
# The same for other dfs
filtered_posting_df['description'] = filtered_posting_df['description'].str.lower()
filtered_posting_df['title'] = filtered_posting_df['title'].str.lower()
filtered_posting_df['skills_desc'] = filtered_posting_df['skills_desc'].str.lower()

/var/folders/jn/000q98496_9_y7jg5hj5hvw40000gn/T/ipykernel_75315/3710244899.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_posting_df['description'] = filtered_posting_df['description'].str.lower()
/var/folders/jn/000q98496_9_y7jg5hj5hvw40000gn/T/ipykernel_75315/3710244899.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_posting_df['title'] = filtered_posting_df['title'].str.lower()
/var/folders/jn/000q98496_9_y7jg5hj5hvw40000gn/T/ipykernel_75315/3710244899.py:8: SettingWithCopyWar

In [6]:
# Joining job_skills_df with skills_df to get the skill name for each job 
joined_job_skills_df = job_skills_df.merge(skills_df, on='skill_abr', how='left')
# Keep the wanted columns
joined_job_skills_df = joined_job_skills_df[['job_id', 'skill_name']]
# Check df
joined_job_skills_df

,job_id,skill_name
0,3884428798,marketing
1,3884428798,public relations
2,3884428798,writing/editing
3,3887473071,sales
4,3887465684,finance
...,...,...
213763,3902876855,human resources
213764,3902878689,management
213765,3902878689,manufacturing
213766,3902883233,sales


In [7]:
# Aggregate the same job_id with skill_name separated by space
agg_joined_job_skills_df = joined_job_skills_df.groupby('job_id', as_index=False).agg(skill_name=('skill_name',' '.join))
agg_joined_job_skills_df

,job_id,skill_name
0,921716,marketing sales
1,1218575,health care provider
2,1829192,health care provider
3,2264355,design art/creative information technology
4,10998357,management manufacturing
...,...,...
126802,3906267117,legal business development
126803,3906267126,engineering information technology
126804,3906267131,sales business development
126805,3906267195,business development sales


In [8]:
# Join joined_job_industries_df and joined_job_skills_df with postings_df
joined_postings_df = filtered_posting_df.merge(agg_joined_job_skills_df,on='job_id', how='left')
joined_postings_df

,job_id,company_id,description,title,skills_desc,skill_name
0,921716,2774458.0,job descriptiona leading real estate firm in n...,marketing coordinator,requirements: \n\nwe are seeking a college or ...,marketing sales
1,1829192,NaN,"at aspen therapy and wellness , we are committ...",mental health therapist/counselor,NaN,health care provider
2,10998357,64896719.0,the national exemplar is accepting application...,assitant restaurant manager,we are currently accepting resumes for foh - a...,management manufacturing
3,23221523,766262.0,senior associate attorney - elder law / trusts...,senior elder law / trusts and estates associat...,this position requires a baseline understandin...,other
4,35982263,NaN,looking for hvac service tech with experience ...,service technician,NaN,information technology
...,...,...,...,...,...,...
123844,3906267117,56120.0,our walnut creek office is currently seeking a...,title ix/investigations attorney,NaN,legal business development
123845,3906267126,1124131.0,about pinterest:\n\nmillions of people across ...,"staff software engineer, ml serving platform",NaN,engineering information technology
123846,3906267131,90552133.0,company overview\n\neps learning is a leading ...,"account executive, oregon/washington",NaN,sales business development
123847,3906267195,2793699.0,the business development manager is a 'hunter'...,business development manager,NaN,business development sales


### Step 2 : Searching for jobs that are related to AI/ML domain
- First, we perform a quick and raw filtering by checking if the job `description` mentioned 'AI', 'ML', 'Artificial Intelligence' or 'Machine Learning' at all;
- However, this method might bring in jobs that are in AI/ML domain but job roles not revelant to AI/ML skills, such as administrations, sales, operations etc. Therefore, we use two other columns---`title` and `skill_name` to get a more accurate result.
- We define functions to give `relevance_score` to each column that contains different `key_words`. Each column's `key_word` increase by different degree so that there is a change of amplitude of each `key_word`.


In [9]:
# Filter out the rows that mentioned 'AI', 'ML', 'Artificial Intelligence' or 'Machine Learning' in `description` column
ai_filtered_df = joined_postings_df[joined_postings_df['description'].str.contains(' ai |artificial intelligence| ml |machine learning', case=False, na=False, regex=True)]
ai_filtered_df

,job_id,company_id,description,title,skills_desc,skill_name
6,103254301,81942316.0,company descriptionraw cereal is a creative de...,producer,NaN,design art/creative information technology
175,3578562287,80035487.0,job description: swahili teacher (contract pos...,swahili instructor,NaN,NaN
266,3728459637,91326370.0,principal backend engineer - join hirebus and ...,principal backend engineer,NaN,engineering information technology
333,3784120102,28154819.0,company descriptionotb english offers business...,artificial intelligence engineer intern - chatbot,NaN,NaN
354,3794986432,89493362.0,"general role:\nwe are hiring for engineers, ma...",engineers / marketing / various,NaN,NaN
...,...,...,...,...,...,...
123521,3906259271,3056912.0,job description:\n\nare you obsessed with data...,director of product (integrations),NaN,product management marketing
123600,3906260032,1288.0,yahoo sports connects fans to the sports and f...,principal technical program manager,NaN,project management information technology
123727,3906261179,11568.0,overview\n\nthe credit risk & decision science...,principal data scientist (credit risk & decisi...,NaN,engineering information technology
123831,3906265301,165654.0,job type\n\nterm (fixed term)\n\nrand is seeki...,bachelor’s level research assistant - quantita...,NaN,research quality assurance science


In [10]:
# Check the missing value of each column
ai_filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3554 entries, 6 to 123845
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   job_id       3554 non-null   int64  
 1   company_id   3530 non-null   float64
 2   description  3554 non-null   object 
 3   title        3554 non-null   object 
 4   skills_desc  36 non-null     object 
 5   skill_name   3494 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 194.4+ KB


In [11]:
# Fill in the missing values with 'n/a'
filled_ai_filtered_df = ai_filtered_df.fillna('n/a')
filled_ai_filtered_df

,job_id,company_id,description,title,skills_desc,skill_name
6,103254301,81942316.0,company descriptionraw cereal is a creative de...,producer,n/a,design art/creative information technology
175,3578562287,80035487.0,job description: swahili teacher (contract pos...,swahili instructor,n/a,n/a
266,3728459637,91326370.0,principal backend engineer - join hirebus and ...,principal backend engineer,n/a,engineering information technology
333,3784120102,28154819.0,company descriptionotb english offers business...,artificial intelligence engineer intern - chatbot,n/a,n/a
354,3794986432,89493362.0,"general role:\nwe are hiring for engineers, ma...",engineers / marketing / various,n/a,n/a
...,...,...,...,...,...,...
123521,3906259271,3056912.0,job description:\n\nare you obsessed with data...,director of product (integrations),n/a,product management marketing
123600,3906260032,1288.0,yahoo sports connects fans to the sports and f...,principal technical program manager,n/a,project management information technology
123727,3906261179,11568.0,overview\n\nthe credit risk & decision science...,principal data scientist (credit risk & decisi...,n/a,engineering information technology
123831,3906265301,165654.0,job type\n\nterm (fixed term)\n\nrand is seeki...,bachelor’s level research assistant - quantita...,n/a,research quality assurance science


In [12]:
# Function that give weights to the `skill` column
def calculate_relevance(description, topic_keywords):
    relevance_score = 0
    for keyword in topic_keywords:
        if keyword in description:
            relevance_score += 1  # Increment score for each matching keyword
    return relevance_score

In [13]:
# Function that give weights to the `title` column
def title_calculate_relevance(description, topic_keywords):
    relevance_score = 0
    for keyword in topic_keywords:
        if keyword in description:
            relevance_score += 2  # Increment score for each matching keyword
    return relevance_score

In [14]:
# Function that give weights to the `description` column
def ai_calculate_relevance(description, topic_keywords):
    relevance_score = 0
    for keyword in topic_keywords:
        if keyword in description:
            relevance_score += 3  # Increment score for each matching keyword
    return relevance_score

In [15]:
# Calculate the score of each row depending on the frequecy of seeing 'AI/ML' related description
descriptions = []
for i in range(len(filled_ai_filtered_df[['description']])):
    description = filled_ai_filtered_df[['description']].iloc[i,0]
    descriptions.append(description)
    
# Calculate relevance scores for each description
description_keywords = ['artificial intelligence', 'machine learning', 'ai', 'ml','llm','llms','nlp']
description_relevance_scores = [(ai_calculate_relevance(description, description_keywords)) for description in descriptions]

In [16]:
# Calculate the score of each row depending on the frequecy of seeing 'AI/ML' related title
titles = []
for i in range(len(filled_ai_filtered_df[['title']])):
    title = filled_ai_filtered_df[['title']].iloc[i,0]
    titles.append(title)
    
# Calculate relevance scores for each description
title_keywords = ['engineer', 'specialist']
title_relevance_scores = [(title_calculate_relevance(description, title_keywords)) for description in titles]

In [17]:
# Calculate the score of each row depending on the frequecy of seeing 'AI/ML' related skills
skills = []
for i in range(len(filled_ai_filtered_df[['skill_name']])):
    skill = filled_ai_filtered_df[['skill_name']].iloc[i,0]
    skills.append(skill)
    
# Calculate relevance scores for each description
skill_keywords = ['engineering']
skill_relevance_scores = [(calculate_relevance(description, skill_keywords)) for description in skills]

In [18]:
# Adding up the scores
total_relevance_score = [x + y + z for x, y, z in zip(description_relevance_scores, skill_relevance_scores, title_relevance_scores)]
total_relevance_score

[3,
 3,
 9,
 14,
 5,
 8,
 3,
 18,
 18,
 12,
 3,
 3,
 6,
 9,
 5,
 12,
 3,
 6,
 14,
 6,
 12,
 9,
 3,
 6,
 7,
 6,
 6,
 13,
 12,
 9,
 6,
 6,
 3,
 3,
 8,
 3,
 9,
 11,
 11,
 5,
 6,
 6,
 9,
 8,
 6,
 6,
 12,
 6,
 3,
 11,
 12,
 6,
 15,
 6,
 10,
 9,
 6,
 9,
 6,
 9,
 6,
 7,
 6,
 8,
 6,
 6,
 6,
 6,
 11,
 6,
 6,
 5,
 3,
 9,
 9,
 9,
 8,
 12,
 6,
 12,
 6,
 9,
 10,
 9,
 3,
 9,
 12,
 8,
 3,
 7,
 3,
 9,
 10,
 3,
 12,
 9,
 3,
 8,
 12,
 12,
 6,
 9,
 9,
 13,
 9,
 7,
 4,
 9,
 9,
 18,
 9,
 11,
 15,
 6,
 3,
 10,
 12,
 9,
 3,
 6,
 9,
 3,
 6,
 15,
 12,
 13,
 11,
 3,
 12,
 6,
 3,
 3,
 9,
 3,
 3,
 9,
 7,
 6,
 3,
 5,
 9,
 6,
 13,
 8,
 7,
 8,
 7,
 9,
 7,
 11,
 7,
 7,
 8,
 11,
 7,
 7,
 9,
 9,
 9,
 9,
 9,
 3,
 11,
 8,
 10,
 18,
 6,
 6,
 6,
 22,
 9,
 6,
 6,
 9,
 3,
 9,
 11,
 15,
 9,
 9,
 4,
 11,
 9,
 12,
 21,
 9,
 15,
 12,
 6,
 7,
 3,
 3,
 4,
 6,
 9,
 6,
 6,
 12,
 3,
 6,
 6,
 6,
 13,
 6,
 3,
 12,
 6,
 9,
 6,
 15,
 9,
 9,
 6,
 9,
 3,
 15,
 18,
 6,
 6,
 6,
 15,
 10,
 11,
 10,
 8,
 11,
 7,
 3,
 6,
 8,
 3,
 6,
 13,
 12,
 

In [19]:
# Creating a new column in the dataframe for the relevance score
ai_filtered_df['relevance_score'] = total_relevance_score

/var/folders/jn/000q98496_9_y7jg5hj5hvw40000gn/T/ipykernel_75315/415793.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ai_filtered_df['relevance_score'] = total_relevance_score


In [20]:
# Sort the `ai_filtered_df` by descending `relevance_score`
desc_ai_filtered_df = ai_filtered_df.sort_values('relevance_score', ascending=False)
desc_ai_filtered_df

,job_id,company_id,description,title,skills_desc,skill_name,relevance_score
83960,3904092471,3618960.0,company profilequantiphi is an award-winning a...,senior machine learning engineer,NaN,engineering,24
51253,3901387863,99331377.0,job title: ai ml / artificial intelligence and...,ai ml software engineer - artificial integence...,NaN,engineering information technology,24
42270,3899532438,1038.0,genai senior developer - senior solution speci...,data scientist - genai - solution specialist -...,NaN,management strategy/planning consulting,23
95865,3904938734,7684.0,ss&c is a global provider of investment and fi...,llm data scientist,NaN,engineering information technology,22
106560,3905327729,23718587.0,direct apply: https://pivotal.bamboohr.com/car...,senior ai data scientist,NaN,engineering information technology,22
...,...,...,...,...,...,...,...
92210,3904578123,1038250.0,"about inductev: at inductev, we're revolutioni...",director of channel partners,NaN,business development general business sales,3
42369,3899534062,22581.0,sia partners is a next-generation management c...,audit consultant - financial services industry,NaN,consulting,3
42011,3899529268,2677.0,mcdonald’s evolving accelerating the arches gr...,"senior technical product manager, restaurant i...",NaN,information technology,3
41872,3899527929,22581.0,sia partners is a next-generation management c...,audit senior consultant - financial services i...,NaN,consulting,3


In [21]:
# Observing the top scores job title
desc_ai_filtered_df[['title', 'relevance_score']].head(2500)

,title,relevance_score
83960,senior machine learning engineer,24
51253,ai ml software engineer - artificial integence...,24
42270,data scientist - genai - solution specialist -...,23
95865,llm data scientist,22
106560,senior ai data scientist,22
...,...,...
110448,"vp, product management ai - gtm",6
32080,data scientist - ai investment,6
31965,technical account manager,6
29377,territory account director (mountain west),6


In [22]:
# Observing the bottom scores job title
desc_ai_filtered_df[['title', 'relevance_score']].tail(585)

,title,relevance_score
2380,business intelligence engineer ii,5
43535,"sr. microsoft presales product specialist, sec...",5
62400,bi data engineer,5
16610,regional solutions specialist - mn/nd/sd/ia,5
60723,senior/lead software engineer,5
...,...,...
92210,director of channel partners,3
42369,audit consultant - financial services industry,3
42011,"senior technical product manager, restaurant i...",3
41872,audit senior consultant - financial services i...,3


### Step 3: Final dataset
- As we can see, when the relevance_score is 3, the job title seems to be less related to AI/ML. Therefore we slice the dataset at relevance_score is 3.

In [23]:
# We select the rows that has `relevance_score` greater than 3
final_ai_filtered_df = ai_filtered_df[ai_filtered_df['relevance_score']>3]
final_ai_filtered_df

,job_id,company_id,description,title,skills_desc,skill_name,relevance_score
266,3728459637,91326370.0,principal backend engineer - join hirebus and ...,principal backend engineer,NaN,engineering information technology,9
333,3784120102,28154819.0,company descriptionotb english offers business...,artificial intelligence engineer intern - chatbot,NaN,NaN,14
354,3794986432,89493362.0,"general role:\nwe are hiring for engineers, ma...",engineers / marketing / various,NaN,NaN,5
520,3848960304,82669556.0,symbolica is building a new foundation for lar...,senior machine learning research engineer,NaN,NaN,8
631,3867234681,87222774.0,we are at the forefront of revolutionizing the...,automation specialist,NaN,engineering information technology,18
...,...,...,...,...,...,...,...
123521,3906259271,3056912.0,job description:\n\nare you obsessed with data...,director of product (integrations),NaN,product management marketing,6
123600,3906260032,1288.0,yahoo sports connects fans to the sports and f...,principal technical program manager,NaN,project management information technology,6
123727,3906261179,11568.0,overview\n\nthe credit risk & decision science...,principal data scientist (credit risk & decisi...,NaN,engineering information technology,7
123831,3906265301,165654.0,job type\n\nterm (fixed term)\n\nrand is seeki...,bachelor’s level research assistant - quantita...,NaN,research quality assurance science,9
